#Mount drive

In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


#Installations and imports

In [4]:
# !pip install scikit-surprise
# !pip install catboost
!pip install xgboost

In [5]:
import os
import pandas as pd
import xgboost
from xgboost import XGBClassifier

#Get Data

In [6]:
home_path = "/content/drive/MyDrive/RS_Kaggle_Competition"

In [7]:
# data of 10 times rows number of any value of user_target_recs

train_data_path = "/content/drive/MyDrive/RS_Kaggle_Competition/train_val_with_distribution/train.csv"
sample_train_data = pd.read_csv(train_data_path, index_col=0)

/usr/local/lib/python3.7/dist-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [8]:
train_data = sample_train_data.fillna("Unknown")
# train_data = sample_train_data.dropna(inplace=False)

train_data["page_view_start_time"] = pd.to_datetime(train_data["page_view_start_time"], unit='ms')
# train_data["month"] = train_data["page_vies_start_time"].dt.month
train_data["day"] = train_data["page_view_start_time"].dt.day

# train_data.drop(columns=["page_view_start_time", "gmt_offset", "user_id_hash", "day"], inplace=True)
train_data.drop(columns=["user_id_hash", "page_view_start_time", "gmt_offset"], inplace=True)

In [9]:
# validation data of 4 times rows number of any value of user_target_recs

val_data_path = "/content/drive/MyDrive/RS_Kaggle_Competition/train_val_with_distribution/val.csv"
sample_val_data = pd.read_csv(val_data_path, index_col=0)

/usr/local/lib/python3.7/dist-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [10]:
val_data = sample_val_data.fillna("Unknown")
# val_data = sample_val_data.dropna(inplace=False)

val_data["page_view_start_time"] = pd.to_datetime(val_data["page_view_start_time"], unit='ms')
# train_data["month"] = train_data["page_vies_start_time"].dt.month
val_data["day"] = val_data["page_view_start_time"].dt.day

# val_data.drop(columns=["page_view_start_time", "gmt_offset", "user_id_hash", "day"], inplace=True)
val_data.drop(columns=["user_id_hash", "page_view_start_time", "gmt_offset"], inplace=True)

In [11]:
test_data = pd.read_csv("/content/drive/MyDrive/RS_Kaggle_Competition/test/test_file.csv")


In [12]:
test_data = test_data.fillna("Unknown")
test_data = test_data.iloc[:,:-1]
test_data["page_view_start_time"] = pd.to_datetime(test_data["page_view_start_time"], unit='ms')
# train_data["month"] = train_data["page_vies_start_time"].dt.month
test_data["day"] = test_data["page_view_start_time"].dt.day

# test_data.drop(columns=["page_view_start_time", "gmt_offset", "user_id_hash", "day"], inplace=True)
test_data.drop(columns=["user_id_hash","page_view_start_time", "gmt_offset"], inplace=True)

#Incremental LabelEncoder - CategoryEncoder

Using a class of inremental labelEncoding - lets us fit multiple times to multiple dataframes - fit and transform to the train, validation and test on all the categorical features

In [13]:
from collections.abc import Iterable

class CategoryEncoder(object):
    """
    Once fit method is called, sklearn.preprocessing.LabelEncoder cannot encode new categories. 
    In this category encoder, fit can be called any number times. It encodes categories which it has not seen before,
    without changing the encoding of existing categories. 
    
    Usually the first category has encoded value of zero. We can override it with value 'start'
    """
    mapping = {}
    start = 0
    
    def __init__(self, start=0):
        self.start = start
        
    def fit(self, l):
        if not isinstance(l, Iterable):
            l = [l]
        for o in l:
            if o not in self.mapping.keys():
                self.mapping[o] = len(self.mapping.keys()) + self.start
        return self
                
    def transform(self, l):
        if isinstance(l, Iterable):
            return [self.mapping.get(o,-1) for o in l]
        else:
            return self.mapping.get(l,-1)
    
    def fit_transform(self,l):
        self.fit(l)
        return self.transform(l) 

In [14]:
encode_cols = ["target_id_hash", "syndicator_id_hash", "campaign_id_hash", "target_item_taxonomy", "placement_id_hash", "publisher_id_hash", "source_id_hash", "source_item_type", "browser_platform", "os_family", "country_code", "region"]

# label encode
def get_encoders_dict(encode_cols=encode_cols):
  encoders_dict = {}
  for encode_col in encode_cols:
    encoders_dict[encode_col] = CategoryEncoder()
  
  return encoders_dict

def encode_data(df, encoders_dict):
  for key in encoders_dict.keys():
    df[key] = encoders_dict[key].fit_transform(df[key].astype(str))
  
  return df

In [15]:
encoders_dict = get_encoders_dict()

In [16]:
train_data = encode_data(train_data, encoders_dict)
val_data = encode_data(val_data, encoders_dict)
test_data = encode_data(test_data, encoders_dict)

#Train Model

In [19]:
model = XGBClassifier()

In [17]:
X_train = train_data.drop(columns="is_click", inplace=False)
y_train = train_data["is_click"]

X_val = val_data.drop(columns="is_click", inplace=False)
y_val = val_data["is_click"]

In [23]:
model.fit(X_train, y_train, eval_metric='auc', eval_set=[(X_train, y_train), (X_val, y_val)])

[0]	validation_0-auc:0.694451	validation_1-auc:0.694489
[1]	validation_0-auc:0.711563	validation_1-auc:0.711721
[2]	validation_0-auc:0.717608	validation_1-auc:0.71766
[3]	validation_0-auc:0.723213	validation_1-auc:0.723526
[4]	validation_0-auc:0.734612	validation_1-auc:0.734834
[5]	validation_0-auc:0.73596	validation_1-auc:0.736269
[6]	validation_0-auc:0.740875	validation_1-auc:0.741162
[7]	validation_0-auc:0.743778	validation_1-auc:0.744112
[8]	validation_0-auc:0.74534	validation_1-auc:0.745628
[9]	validation_0-auc:0.74652	validation_1-auc:0.746809
[10]	validation_0-auc:0.750379	validation_1-auc:0.750718
[11]	validation_0-auc:0.761658	validation_1-auc:0.761926
[12]	validation_0-auc:0.763416	validation_1-auc:0.76368
[13]	validation_0-auc:0.769348	validation_1-auc:0.7696
[14]	validation_0-auc:0.771408	validation_1-auc:0.771687
[15]	validation_0-auc:0.773831	validation_1-auc:0.774149
[16]	validation_0-auc:0.77919	validation_1-auc:0.779452
[17]	validation_0-auc:0.781957	validation_1-auc:0

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

The results we received from the xgboost model weren't better than the ones that we received on the same validation data from other models (up to auc=~0.858) so we did not submit it to the kaggle competition